In [ ]:
import os
import csv
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub

def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    image = image.resize((224, 224), Image.BICUBIC)
    image_array = np.array(image, dtype=np.float32) / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

def load_images_and_labels(data_directory):
    images, labels = [], []
    list_of_people = sorted([name for name in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, name)) and name != '.ipynb_checkpoints'])

    for label, person in enumerate(list_of_people):
        person_folder = os.path.join(data_directory, person)

        for image_filename in os.listdir(person_folder):
            if image_filename == '.ipynb_checkpoints' or not image_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            image_path = os.path.join(person_folder, image_filename)
            image_array = preprocess_image(image_path)
            images.append(image_array)
            labels.append(label)

    images = np.concatenate(images, axis=0)
    labels = np.array(labels)

    return images, labels, list_of_people

data_directory = '/content/drive/MyDrive/faces-old'
images, labels, list_of_people = load_images_and_labels(data_directory)

num_classes = len(list_of_people)

# Use MobileNetV2 model
mobilenet_v2_model = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", output_shape=[1280], trainable=False)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
    mobilenet_v2_model,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Split the data into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)

# Train the model
history = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=20, batch_size=32)




Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 1s 550ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 1s 610ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 1s 951ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 1s 857ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/20
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/20
1/1 [==============================] - 1s 582ms/step - loss

In [ ]:
def recognize_person(image_path, model, list_of_people):
    input_image = preprocess_image(image_path)
    predictions = model.predict(input_image)
    predicted_class_index = np.argmax(predictions[0])
    return list_of_people[predicted_class_index]

# Load a sample image for inference
image_path = "/content/drive/MyDrive/reader-test-person.png"
recognized_person = recognize_person(image_path, model, list_of_people)
print("The person in the picture is:", recognized_person)



1/1 [==============================] - 1s 1s/step
The person in the picture is: Logan


In [ ]:
# Define the representative dataset
def representative_dataset():
    for i in range(len(train_images)):
        yield([train_images[i].reshape(1, 224, 224, 3)])

# Convert the model to a quantized TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()

# Save the quantized TensorFlow Lite model to a file
with open('/content/drive/MyDrive/face_recognition_quant.tflite', 'wb') as f:
    f.write(tflite_quant_model)

# Save the list_of_people to a CSV file
with open('/content/drive/MyDrive/list_of_people.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for person in list_of_people:
        writer.writerow([person])

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  98481      0 --:--:-- --:--:-- --:--:-- 98481
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:5 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,332 B]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 https://packages.cloud.google.com/apt coral-edgetpu-stable/main amd64 P

In [ ]:
!edgetpu_compiler /content/drive/MyDrive/face_recognition_quant.tflite

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 1367 ms.

Input model: /content/drive/MyDrive/face_recognition_quant.tflite
Input size: 2.74MiB
Output model: face_recognition_quant_edgetpu.tflite
Output size: 2.94MiB
On-chip memory used for caching model parameters: 2.79MiB
On-chip memory remaining for caching model parameters: 4.89MiB
Off-chip memory used for streaming uncached model parameters: 64.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 69
Operation log: face_recognition_quant_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 
